In [10]:
import pandas as pd
import numpy as np


sales = pd.read_csv('m5-forecasting-accuracy/sales_train_validation.csv')
calender = pd.read_csv('m5-forecasting-accuracy/calendar.csv')
prices = pd.read_csv('m5-forecasting-accuracy/sell_prices.csv')
# -------------------------------
# Step 1: Subset the data
# -------------------------------
subset = sales[
    (sales['dept_id'].isin(['FOODS_1','FOODS_2'])) &
    (sales['store_id'].isin(['CA_1','CA_2']))
]

# Melt wide -> long
df_long = subset.melt(
    id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'],
    var_name='d', value_name='sales'
)

# Merge with calendar to get actual dates and event info
df_long = df_long.merge(
    calender[['d','date','weekday','month','year','event_name_1']],
    on='d', how='left'
)

# Convert 'date' to datetime
df_long['date'] = pd.to_datetime(df_long['date'], errors='coerce')
df_long = df_long.dropna(subset=['date'])  # drop invalid dates

# -------------------------------
# Step 2: Aggregate daily sales by store
# -------------------------------
daily_sales = df_long.groupby(['date','store_id'])['sales'].sum().reset_index()

# Step 3: Ensure continuous dates per store (fixed)
all_dates = pd.date_range(daily_sales['date'].min(), daily_sales['date'].max())
stores = daily_sales['store_id'].unique()

dfs = []
for store in stores:
    temp = daily_sales[daily_sales['store_id'] == store].set_index('date')
    temp = temp.reindex(all_dates, fill_value=0)
    temp['store_id'] = store
    temp = temp.reset_index().rename(columns={'index':'date'})
    dfs.append(temp)

daily_sales = pd.concat(dfs, ignore_index=True)


# -------------------------------
# Step 4: Feature engineering
# -------------------------------
daily_sales['day_of_week'] = daily_sales['date'].dt.dayofweek
daily_sales['month'] = daily_sales['date'].dt.month
daily_sales['week_of_year'] = daily_sales['date'].dt.isocalendar().week
daily_sales['year'] = daily_sales['date'].dt.year

# Event/holiday flag
df_events = df_long[['date','event_name_1']].drop_duplicates()
daily_sales = daily_sales.merge(df_events, on='date', how='left')
daily_sales['event_flag'] = daily_sales['event_name_1'].notna().astype(int)

# Rolling averages
daily_sales['rolling_7'] = daily_sales.groupby('store_id')['sales'].transform(lambda x: x.rolling(7).mean())
daily_sales['rolling_14'] = daily_sales.groupby('store_id')['sales'].transform(lambda x: x.rolling(14).mean())

# Optional: log-transform to stabilize variance
daily_sales['sales_log'] = np.log1p(daily_sales['sales'])

# -------------------------------
# Step 5: Save prepared dataset
# -------------------------------
daily_sales.to_csv("prepared_sales.csv", index=False)

print("02_data_preparation complete. Dataset ready for forecasting!")
daily_sales.head()


02_data_preparation complete. Dataset ready for forecasting!


,date,store_id,sales,day_of_week,month,week_of_year,year,event_name_1,event_flag,rolling_7,rolling_14,sales_log
0,2011-01-29,CA_1,971,5,1,4,2011,NaN,0,NaN,NaN,6.879356
1,2011-01-30,CA_1,939,6,1,4,2011,NaN,0,NaN,NaN,6.845880
2,2011-01-31,CA_1,610,0,1,5,2011,NaN,0,NaN,NaN,6.415097
3,2011-02-01,CA_1,651,1,2,5,2011,NaN,0,NaN,NaN,6.480045
4,2011-02-02,CA_1,536,2,2,5,2011,NaN,0,NaN,NaN,6.285998


In [11]:
daily_sales.tail()

,date,store_id,sales,day_of_week,month,week_of_year,year,event_name_1,event_flag,rolling_7,rolling_14,sales_log
3821,2016-04-20,CA_2,872,2,4,16,2016,NaN,0,919.000000,957.857143,6.771936
3822,2016-04-21,CA_2,683,3,4,16,2016,NaN,0,911.000000,947.357143,6.527958
3823,2016-04-22,CA_2,975,4,4,16,2016,NaN,0,912.285714,953.928571,6.883463
3824,2016-04-23,CA_2,1243,5,4,16,2016,NaN,0,937.714286,945.571429,7.126087
3825,2016-04-24,CA_2,1302,6,4,16,2016,NaN,0,951.142857,932.785714,7.172425


In [12]:
daily_sales.isnull().sum()

date               0
store_id           0
sales              0
day_of_week        0
month              0
week_of_year       0
year               0
event_name_1    3518
event_flag         0
rolling_7         12
rolling_14        26
sales_log          0
dtype: int64